# Model Exercises
Using the titanic data, in your classification-exercises repository, create a notebook, model.ipynb where you will do the following:

In [4]:
import pandas as pd
import numpy as np
import os
from pydataset import data

import seaborn as sns
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

from sklearn.metrics import classification_report

from env import get_db_url
from acquire import new_titanic_data
from acquire import get_titanic_data

In [5]:
# Decision Tree and Model Evaluation Imports
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix

In [6]:
titanic_df = get_titanic_data()
titanic_df.head()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,3,male,22.0,1,0,7.2500,S,Third,NaN,Southampton,0
1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,1,3,female,26.0,0,0,7.9250,S,Third,NaN,Southampton,1
3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,0,3,male,35.0,0,0,8.0500,S,Third,NaN,Southampton,1


### 1. What is your baseline prediction? What is your baseline accuracy? remember: your baseline prediction for a classification problem is predicting the most prevelant class in the training dataset (the mode). When you make those predictions, what is your accuracy? This is your baseline accuracy.

In [7]:
titanic_df.survived.value_counts()

0    549
1    342
Name: survived, dtype: int64

In [9]:
titanic_df['survived_baseline'] = '0'
titanic_df.head()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone,baseline,survived_baseline
0,0,0,3,male,22.0,1,0,7.2500,S,Third,NaN,Southampton,0,0,0
1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0,0,0
2,2,1,3,female,26.0,0,0,7.9250,S,Third,NaN,Southampton,1,0,0
3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0,0,0
4,4,0,3,male,35.0,0,0,8.0500,S,Third,NaN,Southampton,1,0,0


### 2. Fit the decision tree classifier to your training sample and transform (i.e. make predictions on the training sample)

### 3. Evaluate your in-sample results using the model score, confusion matrix, and classification report.

### 4. Compute: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

### 5. Run through steps 2-4 using a different ```max_depth``` value.

### 6. Which model performs better on your in-sample data?

### 7. Which model performs best on your out-of-sample data, the ```validate``` set?

# EXTRA

### 1. Work through these same exercises using the Telco dataset.

### 2. Experiment with this model on other datasets with a higher number of output classes.